# Importa le librerie necessarie


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix
import seaborn as sns
from tensorflow.keras.datasets import mnist

# Impostazione dei semi casuali per riproducibilità
np.random.seed(42)
tf.random.set_seed(42)

# Design, training e validazione di un classificatore per MNIST

questo classificatore (che può essere simile o uguale) a quello che hai già allenato per la prima parte della tesi, dovrà essere addestrato in questa fase e poi dovrà andare a valutare le cifre generate dal generatore della GAN nella seconda fase.


In [ ]:
# Funzione per costruire il classificatore MNIST

def build_classifier():
    model = keras.Sequential([

    # istruzione: riempi con architettura

    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
# Carica e prepara i dati MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalizza i dati come nel codice originale
x_train = (x_train.astype(np.float32) - 127.5) / 127.5
x_test = (x_test.astype(np.float32) - 127.5) / 127.5
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

In [ ]:
# Crea e addestra il classificatore
MNIST_classifier = build_classifier()

history = MNIST_classifier.fit(x_train, y_train, epochs=10, validation_split=0.2, verbose=1, batch_size=256)

In [ ]:
# Visualizza l'andamento dell'addestramento
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Valuta il classificatore sul set di test
test_loss, test_acc = MNIST_classifier.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")

In [ ]:
# Crea la matrice di confusione
y_pred = MNIST_classifier.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Visualizza la matrice di confusione
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.show()

In [ ]:
# Visualizza alcuni esempi con le probabilità predette

num_examples = 10
random_indices = np.random.choice(len(x_test), num_examples, replace=False)

plt.figure(figsize=(15, 3*num_examples))
for i, idx in enumerate(random_indices):
    plt.subplot(num_examples, 2, 2*i+1)
    plt.imshow(x_test[idx].reshape(28, 28), cmap='gray')
    plt.title(f"True Label: {y_test[idx]}")
    plt.axis('off')

    plt.subplot(num_examples, 2, 2*i+2)
    probs = MNIST_classifier.predict(x_test[idx].reshape(1, 28, 28, 1))[0]
    plt.bar(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], probs)
    plt.title(f"Predicted Probabilities")
    plt.ylim(0, 1)

plt.tight_layout()
plt.show()

# Design e training di una GAN per generare immagini di numeri 1, 2 e 3 simili a MNIST

Per semplificare la task andremo a generare solo numeri 1, 2 e 3.


In [ ]:
# Impostazione parametri globali

LATENT_DIM = 100 # Dimensione del vettore di rumore latente da usare come input al generatore
NUM_EPOCHS = 100 # Numero di epoche di allenamento del GAN
BATCH_SIZE = 64 # Dimensione del batch per l'allenamento
NUM_SAMPLES = 5000 # Numero di esempi da selezionare casualmente dal dataset MNIST (evitiamo di usare l'intero dataset per risparmiare tempo)

In [ ]:
# # # Funzioni di utilità per la preparazione dei dati e la visualizzazione

def carica_e_prepara_dati():
    """
    Carica il dataset MNIST, seleziona solo le cifre 1, 2 e 3,
    normalizza le immagini e seleziona un sottoinsieme casuale.
    """
    (x_train, y_train), (_, _) = mnist.load_data()

    # Seleziona solo le cifre 1, 2 e 3
    mask = np.isin(y_train, [1, 2, 3])
    x_train, y_train = x_train[mask], y_train[mask]

    # Normalizza le immagini nell'intervallo [-1, 1]
    x_train = (x_train.astype(np.float32) - 127.5) / 127.5
    x_train = np.expand_dims(x_train, axis=-1)

    # Seleziona un sottoinsieme casuale di esempi
    indici_casuali = np.random.choice(x_train.shape[0], NUM_SAMPLES, replace=False)
    x_train, y_train = x_train[indici_casuali], y_train[indici_casuali]

    return x_train, y_train

def visualizza_esempi(x_train, num_esempi=10):
    """
    Visualizza un numero specificato di esempi casuali dal dataset.
    """
    num_righe = int(np.sqrt(num_esempi))
    num_colonne = int(np.ceil(num_esempi / num_righe))

    plt.figure(figsize=(num_colonne, num_righe))
    for i in range(num_esempi):
        plt.subplot(num_righe, num_colonne, i+1)
        plt.imshow(x_train[np.random.randint(0, x_train.shape[0])].reshape(28, 28), cmap='gray')
        plt.axis('off')
    plt.show()


# Carica e prepara i dati
x_train, y_train = carica_e_prepara_dati()

print("Dimensioni del dataset di allenamento:", x_train.shape)

visualizza_esempi(x_train, num_esempi=100)

In [ ]:
# # # definizione del generatore e il discriminatore del GAN e le funzioni di perdita

def crea_generatore():
    """
    Crea e restituisce il modello del generatore.
    """
    model = keras.Sequential([

    # istruzione: progettare architettura per il generatore

    ])
    return model

def crea_discriminatore():
    """
    Crea e restituisce il modello del discriminatore.
    """
    model = keras.Sequential([

        # istruzione: progettare architettura per il discriminatore
        # l'ultimo strato sarà un layer denso con un valore in output, che verrà successivamente
        # interpretato come probabilità di essere una sorgente reale o fake

        layers.Dense(1, activation='linear')
    ])
    return model

def loss_discriminatore(output_reale, output_falso):
    """
    Calcola la loss del discriminatore.
    Quanto bene il discriminatore riesce a distinguere le immagini reali da quelle generate?
    Questa loss cerca di massimizzare la probabilità di classificare correttamente sia le immagini reali che quelle generate.
    In altre parole, cerca di minimizzare la probabilità che il discriminatore classifichi le immagini reali come false e le immagini generate come vere.
    """
    cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)
    loss_reale = cross_entropy(tf.ones_like(output_reale), output_reale)
    loss_falso = cross_entropy(tf.zeros_like(output_falso), output_falso)
    return loss_reale + loss_falso

def loss_generatore(output_falso):
    """
    Calcola la loss del generatore.
    Quanto bene il generatore riesce a ingannare il discriminatore?
    Questa loss cerca di minimizzare la probabilità che il discriminatore classifichi correttamente le immagini generate come false.
    In altre parole, cerca di massimizzare la probabilità che il discriminatore classifichi le immagini generate come vere.
    """
    cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)
    return cross_entropy(tf.ones_like(output_falso), output_falso)


# Creazione dei modelli e visualizzazione dettagli tramite la funzione .summary() di Keras
generatore = crea_generatore()
generatore.summary()
discriminatore = crea_discriminatore()
discriminatore.summary()


In [ ]:

@tf.function # Questo comando vuol dire che la funzione verrà eseguita in modalità grafica per prestazioni migliori durante l'allenamento del GAN (una peculiarità di TensorFlow)
def step_allenamento(immagini, generatore, discriminatore, ottimizzatore_gen, ottimizzatore_disc):
    """
    Esegue un singolo step di allenamento per il GAN.
    Questa funzione alterna l'ottimizzazione del generatore e del discriminatore, calcolando le relative loss.
    L'allenamento del generatore cerca di massimizzare la probabilità che il discriminatore classifichi le immagini generate come vere.
    L'allenamento del discriminatore cerca di massimizzare la probabilità di classificare correttamente sia le immagini reali che quelle generate.
    """
    rumore = tf.random.normal([BATCH_SIZE, LATENT_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        immagini_generate = generatore(rumore, training=True)

        output_reale = discriminatore(immagini, training=True)
        output_falso = discriminatore(immagini_generate, training=True)

        loss_gen = loss_generatore(output_falso)
        loss_disc = loss_discriminatore(output_reale, output_falso)

    gradienti_generatore = gen_tape.gradient(loss_gen, generatore.trainable_variables)
    gradienti_discriminatore = disc_tape.gradient(loss_disc, discriminatore.trainable_variables)

    ottimizzatore_gen.apply_gradients(zip(gradienti_generatore, generatore.trainable_variables))
    ottimizzatore_disc.apply_gradients(zip(gradienti_discriminatore, discriminatore.trainable_variables))

    return loss_gen, loss_disc

def salva_immagini_generate(immagini, epoca, cartella='immagini_generate'):
    """
    Salva le immagini generate in una griglia.
    Questo serve per visualizzare come cambiano le immagini generate durante l'allenamento.
    Se le immagini vengono generate a partire dallo stesso rumore, dovrebbe vedersi un progressivo miglioramento nella qualità delle immagini durante l'allenamento.
    """
    import os

    fig, axs = plt.subplots(8, 8, figsize=(8, 8))
    axs = axs.flatten()

    for img, ax in zip(immagini, axs):
        ax.imshow((img.numpy() * 127.5 + 127.5).astype(np.uint8), cmap='gray')
        ax.axis('off')

    if not os.path.exists(cartella):
        os.makedirs(cartella)

    plt.savefig(f"{cartella}/epoca_{epoca+1:03d}.png")
    plt.close()

def valuta_frazione_immagini_confidenti(generatore, classificatore, confidenza = 0.95, num_immagini=100):
    """
    Valuta la frazione di immagini generate classificate come 1, 2, o 3 con alta confidenza.
    Serve a valutare quanto bene il generatore è in grado di generare immagini simili a quelle del dataset.
    In questo modo possiamo vedere se il generatore è in grado di generare immagini simili a quelle del dataset di allenamento.
    Si potrebbe fare visulmente, ma questa funzione fornisce una misura quantitativa.
    Fatto questo, useremo anche LLMs con visione per valutare la qualità delle immagini generate in modo indipendente.
    """

    # istruzione: riempi questa funzione con del codice che genera un numero num_immagini di immagini
    # usando il generatore, e le passa in input per la classificazione al classificatore MNIST_classifier.
    # Bisogna poi calcolare la frazione di queste immagini per cui il MNIST_classifier pensa che l'immagine sia
    # un 1, o un 2 o un 3, con una probabilità piu grande della variabile "confidenza".

    return conteggio_confidenti


In [ ]:
def salva_immagini_random_generate(generatore, num_esempi=100, epoca=0, latent_dim=32):
    """
    Visualizza esempi di immagini generate.
    """
    rumore = tf.random.normal([num_esempi, latent_dim])
    immagini_generate = generatore(rumore, training=False).numpy()

    num_righe = int(np.sqrt(num_esempi))
    num_colonne = int(np.ceil(num_esempi / num_righe))

    plt.figure(figsize=(num_colonne, num_righe))
    for i in range(num_esempi):
        plt.subplot(num_righe, num_colonne, i+1)
        plt.imshow(immagini_generate[i, :, :, 0] * 0.5 + 0.5, cmap='gray')
        plt.axis('off')

    plt.tight_layout()
    plt.savefig(f"immagini_casuali/epoca_{epoca+1:03d}.png")
    plt.close()



def allena_gan(generatore, discriminatore, dataset, classificatore):
    """
    Allena il GAN e restituisce le metriche di allenamento.
    """

    # Learning rate per gli ottimizzatori del generatore e del discriminatore (come nell'articolo originale di DCGAN)
    ottimizzatore_gen = keras.optimizers.Adam(1e-4)
    ottimizzatore_disc = keras.optimizers.Adam(1e-4)

    # definire le liste per memorizzare le loss e la frazione di immagini confidenti
    losses_gen, losses_disc = [], []
    conteggio_immagini_confidenti = []

    # Genera rumore fisso per visualizzare le immagini generate durante l'allenamento
    # Lo facciamo per 64 immagini
    rumore_fisso = tf.random.normal([64, LATENT_DIM])

    # comincia l'allenamento, ciclo sul numero di epoche
    for epoca in range(NUM_EPOCHS):

        losses_gen_batch, losses_disc_batch = [], []

        # ciclo sul dataset in batch all'interno di ogni epoca
        for batch in dataset:
            loss_gen, loss_disc = step_allenamento(batch, generatore, discriminatore,
                                                   ottimizzatore_gen, ottimizzatore_disc)
            losses_gen_batch.append(loss_gen)
            losses_disc_batch.append(loss_disc)

        # salviamo le loss medie per epoca, mediando su tutti i batch
        losses_gen.append(np.mean(losses_gen_batch))
        losses_disc.append(np.mean(losses_disc_batch))

        # Genera e salva immagini usando rumore fisso
        immagini_generate = generatore(rumore_fisso, training=False)
        salva_immagini_generate(immagini_generate, epoca)

        # Valuta la frazione di immagini confidenti
        conteggio_confidenti = valuta_frazione_immagini_confidenti(generatore, classificatore)
        conteggio_immagini_confidenti.append(conteggio_confidenti)

        salva_immagini_random_generate(generatore, num_esempi=100, epoca=epoca, latent_dim=LATENT_DIM)

        print(f"Epoca {epoca+1}/{NUM_EPOCHS}, Loss_Gen: {losses_gen[-1]:.4f}, "
              f"Loss_Disc: {losses_disc[-1]:.4f}, Immagini confidenti: {conteggio_confidenti:.2f}")

    return losses_gen, losses_disc, conteggio_immagini_confidenti


# Crea il dataset in formato TensorFlow, diviso in batch e shufflato casualmente
dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(x_train.shape[0]).batch(BATCH_SIZE)

# Allena il GAN e salva le metriche di allenamento e la frazione di immagini confidenti per ogni epoca, da visualizzare successivamente
losses_gen, losses_disc, conteggio_immagini_confidenti = allena_gan(generatore, discriminatore, dataset, classificatore= MNIST_classifier)


In [ ]:
def visualizza_metriche(losses_gen, losses_disc, conteggio_immagini_confidenti):
    """
    Visualizza le metriche di allenamento del GAN.
    """
    epoche = np.arange(1, len(losses_gen)+1)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    ax1.plot(epoche, losses_gen, label='Loss Generatore', color='blue')
    ax1.plot(epoche, losses_disc, label='Loss Discriminatore', color='red')
    ax1.set_xlabel('Epoche')
    ax1.set_ylabel('Loss')
    ax1.set_title('Loss del Generatore e del Discriminatore')
    ax1.legend()

    ax2.plot(epoche, conteggio_immagini_confidenti, label='Immagini confidenti', color='green')
    ax2.set_xlabel('Epoche')
    ax2.set_ylabel('Frazione')
    ax2.set_title('Frazione di Immagini Confidenti')
    ax2.legend()

    plt.tight_layout()
    plt.show()

# Visualizza le metriche di allenamento
visualizza_metriche(losses_gen, losses_disc, conteggio_immagini_confidenti)

In [ ]:
# Visualizza degli esempi di immagini generate durante l'allenamento con le relative probabilità predette dal classificatore pre-addestrato

def visualizza_immagini_generate_con_probabilita(generatore, classificatore, num_esempi=20):
    """
    Visualizza esempi di immagini generate con le relative probabilità predette.
    """
    rumore = tf.random.normal([num_esempi, LATENT_DIM])
    immagini_generate = generatore(rumore, training=False).numpy()

    plt.figure(figsize=(15, 3*num_esempi))
    for i in range(num_esempi):
        plt.subplot(num_esempi, 2, 2*i+1)
        plt.imshow(immagini_generate[i, :, :, 0] * 0.5 + 0.5, cmap='gray')
        plt.axis('off')

        plt.subplot(num_esempi, 2, 2*i+2)
        probs = classificatore.predict(immagini_generate[i].reshape(1, 28, 28, 1))[0]
        plt.bar(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], probs)
        plt.title("Probabilità Predette")
        plt.ylim(0, 1)

    plt.tight_layout()
    plt.show()

# Visualizza esempi di immagini generate
visualizza_immagini_generate_con_probabilita(generatore, classificatore = MNIST_classifier)